In [ ]:
from pyspark.sql.functions import (
    round, col, dayofmonth, month, year, quarter, 
    substring, when, regexp_replace
)

StatementMeta(, deac1652-398f-478e-bc15-19c06096033b, 6, Finished, Available, Finished)

In [ ]:
# Chemin vers la table Bronze
bronze_table_path = "abfss://Eolienne@onelake.dfs.fabric.microsoft.com/LH_Wind_Power_Bronze.Lakehouse/Tables/dbo/wind_power"

# Charger les données
df = spark.read.format("delta").load(bronze_table_path)

# Afficher le schéma et un aperçu
print("📊 Schéma des données Bronze :")
df.printSchema()

print(f"\n📈 Nombre de lignes : {df.count()}")

print("\n🔍 Aperçu des 5 premières lignes :")
df.show(5, truncate=False)

StatementMeta(, deac1652-398f-478e-bc15-19c06096033b, 7, Finished, Available, Finished)

📊 Schéma des données Bronze :
root
 |-- production_id: long (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- time: string (nullable = true)
 |-- turbine_name: string (nullable = true)
 |-- capacity: long (nullable = true)
 |-- location_name: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- region: string (nullable = true)
 |-- status: string (nullable = true)
 |-- responsible_department: string (nullable = true)
 |-- wind_speed: double (nullable = true)
 |-- wind_direction: string (nullable = true)
 |-- energy_produced: double (nullable = true)




📈 Nombre de lignes : 432

🔍 Aperçu des 5 premières lignes :


+-------------+-------------------+--------+------------+--------+-------------+--------+---------+--------+----------------------+----------------------+----------+--------------+---------------+
|production_id|date               |time    |turbine_name|capacity|location_name|latitude|longitude|region  |status                |responsible_department|wind_speed|wind_direction|energy_produced|
+-------------+-------------------+--------+------------+--------+-------------+--------+---------+--------+----------------------+----------------------+----------+--------------+---------------+
|6319         |2024-06-15 00:00:00|15-00-00|Turbine A   |2200    |Location 1   |34.0522 |-118.2437|Region A|Online                |Operations            |22.69586  |N             |808.75332      |
|6320         |2024-06-15 00:00:00|15-00-00|Turbine B   |2000    |Location 2   |36.7783 |-119.4179|Region B|Online                |Operations            |18.33018  |NW            |1237.96877     |
|6321         |

In [ ]:
# Appliquer toutes les transformations en une seule opération chaînée
df_transformed = (df
    # 🔢 Arrondir les valeurs numériques à 2 décimales
    .withColumn("wind_speed", round(col("wind_speed"), 2))
    .withColumn("energy_produced", round(col("energy_produced"), 2))
    
    # 📅 Extraire les composants de date
    .withColumn("day", dayofmonth(col("date")))
    .withColumn("month", month(col("date")))
    .withColumn("quarter", quarter(col("date")))
    .withColumn("year", year(col("date")))
    
    # 🕐 Corriger le format de time (remplacer - par :)
    .withColumn("time", regexp_replace(col("time"), "-", ":"))
    
    # ⏰ Extraire les composants de temps
    .withColumn("hour_of_day", substring(col("time"), 1, 2).cast("int"))
    .withColumn("minute_of_hour", substring(col("time"), 4, 2).cast("int"))
    .withColumn("second_of_minute", substring(col("time"), 7, 2).cast("int"))
    
    # 🌅 Calculer la période de la journée
    .withColumn("time_period", 
        when((col("hour_of_day") >= 5) & (col("hour_of_day") < 12), "Morning")
        .when((col("hour_of_day") >= 12) & (col("hour_of_day") < 17), "Afternoon")
        .when((col("hour_of_day") >= 17) & (col("hour_of_day") < 21), "Evening")
        .otherwise("Night")
    )
)

print("✅ Transformations appliquées avec succès !")

StatementMeta(, deac1652-398f-478e-bc15-19c06096033b, 8, Finished, Available, Finished)

✅ Transformations appliquées avec succès !


In [ ]:
# Afficher un échantillon des données transformées
print("📊 Aperçu des données transformées :")
df_transformed.select(
    "date", "time", "turbine_name", 
    "wind_speed", "energy_produced", 
    "day", "month", "year", "quarter",
    "hour_of_day", "time_period"
).show(10)

print(f"\n📈 Nombre de colonnes : {len(df_transformed.columns)}")
print(f"📋 Nouvelles colonnes ajoutées : day, month, quarter, year, hour_of_day, minute_of_hour, second_of_minute, time_period")

StatementMeta(, deac1652-398f-478e-bc15-19c06096033b, 9, Finished, Available, Finished)

📊 Aperçu des données transformées :


+-------------------+--------+------------+----------+---------------+---+-----+----+-------+-----------+-----------+
|               date|    time|turbine_name|wind_speed|energy_produced|day|month|year|quarter|hour_of_day|time_period|
+-------------------+--------+------------+----------+---------------+---+-----+----+-------+-----------+-----------+
|2024-06-15 00:00:00|00:00:00|   Turbine A|      5.74|        1783.39| 15|    6|2024|      2|          0|      Night|
|2024-06-15 00:00:00|00:00:00|   Turbine B|     23.92|            0.0| 15|    6|2024|      2|          0|      Night|
|2024-06-15 00:00:00|00:00:00|   Turbine C|      5.72|        1651.84| 15|    6|2024|      2|          0|      Night|
|2024-06-15 00:00:00|00:10:00|   Turbine A|     13.02|        1351.88| 15|    6|2024|      2|          0|      Night|
|2024-06-15 00:00:00|00:10:00|   Turbine B|      8.77|         931.99| 15|    6|2024|      2|          0|      Night|
|2024-06-15 00:00:00|00:10:00|   Turbine C|     23.31|  

In [ ]:
from pyspark.sql.functions import count, when, isnan, col, min as spark_min, max as spark_max

# Vérifier qu'il n'y a pas de valeurs nulles dans les colonnes critiques
print("=== 🔍 Vérification des valeurs nulles ===")
null_counts = df_transformed.select([
    count(when(col(c).isNull(), c)).alias(c) 
    for c in ["wind_speed", "energy_produced", "day", "month", "year", "time_period"]
])
null_counts.show()

# Vérifier les valeurs uniques de time_period
print("\n=== 📊 Distribution des périodes de la journée ===")
df_transformed.groupBy("time_period").count().orderBy("count", ascending=False).show()

# Vérifier les plages de dates
print("\n=== 📅 Plage de dates ===")
df_transformed.select(
    spark_min("date").alias("Date minimale"),
    spark_max("date").alias("Date maximale")
).show()

# Statistiques descriptives
print("\n=== 📈 Statistiques sur les mesures ===")
df_transformed.select("wind_speed", "energy_produced").describe().show()

StatementMeta(, deac1652-398f-478e-bc15-19c06096033b, 10, Finished, Available, Finished)

=== 🔍 Vérification des valeurs nulles ===


+----------+---------------+---+-----+----+-----------+
|wind_speed|energy_produced|day|month|year|time_period|
+----------+---------------+---+-----+----+-----------+
|         0|              0|  0|    0|   0|          0|
+----------+---------------+---+-----+----+-----------+


=== 📊 Distribution des périodes de la journée ===


+-----------+-----+
|time_period|count|
+-----------+-----+
|      Night|  144|
|    Morning|  126|
|  Afternoon|   90|
|    Evening|   72|
+-----------+-----+


=== 📅 Plage de dates ===
+-------------------+-------------------+
|      Date minimale|      Date maximale|
+-------------------+-------------------+
|2024-06-15 00:00:00|2024-06-15 00:00:00|
+-------------------+-------------------+


=== 📈 Statistiques sur les mesures ===


+-------+------------------+------------------+
|summary|        wind_speed|   energy_produced|
+-------+------------------+------------------+
|  count|               432|               432|
|   mean|13.803958333333334|1086.8985185185186|
| stddev| 6.719660721473737| 588.6453854989962|
|    min|              2.12|               0.0|
|    max|             24.99|           1995.45|
+-------+------------------+------------------+



In [ ]:
# Chemin vers la table Silver
silver_table_path = "abfss://Eolienne@onelake.dfs.fabric.microsoft.com/LH_Wind_Power_Silver.Lakehouse/Tables/dbo/wind_power"

# Sauvegarder en mode overwrite (écrasement complet)
df_transformed.write.format("delta").mode("overwrite").save(silver_table_path)

print("✅ Données transformées et sauvegardées dans Silver")
print(f"📊 Nombre de lignes sauvegardées : {df_transformed.count()}")

StatementMeta(, deac1652-398f-478e-bc15-19c06096033b, 12, Finished, Available, Finished)

✅ Données transformées et sauvegardées dans Silver
📊 Nombre de lignes sauvegardées : 432


# Transformation Bronze → Silver

## 📋 Objectif
Nettoyer, standardiser et enrichir les données brutes du Lakehouse Bronze.

## 🔄 Transformations appliquées
1. **Nettoyage numérique** : Arrondi à 2 décimales
2. **Enrichissement temporel** : Extraction jour/mois/année/trimestre
3. **Standardisation** : Correction du format de time
4. **Calcul métier** : Période de la journée basée sur l'heure

## 📦 Dépendances
- **Input** : `LH_Wind_Power_Bronze.dbo.wind_power`
- **Output** : `LH_Wind_Power_Silver.dbo.wind_power`

## ⚙️ Mode de sauvegarde
- **Mode** : Overwrite (écrasement complet)
- **Raison** : Simplicité pour ce projet pédagogique